# Philipp's first demo

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

buffer = hv.streams.Buffer(data=pd.DataFrame(np.random.randn(10, 10).cumsum(axis=1), columns=[chr(65+i) for i in range(10)]))
def out(data):
    return hv.NdOverlay(
        {i: hv.Curve(data, 'index', (chr(65+i), 'Amplitude')).opts(subcoordinate_y=True) for i in range(10)}
    )

hv.DynamicMap(out, streams=[buffer]).opts(legend_position='right', width=800)


In [ ]:

for i in range(2, 1000):
    buffer.event(data=pd.DataFrame(np.random.randn(10, 10).cumsum(axis=1), columns=[chr(65+i) for i in range(10)], index=np.arange(i*10, i*10+10)))
    time.sleep(0.1)

# Demetris' version, threaded stream, buttons, inspection plot
- Added an inspection plot (because I haven't figured out yet how to do a return to stream)

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
import panel as pn
from threading import Thread

hv.extension('bokeh')
pn.extension()

initial_data = pd.DataFrame(np.random.randn(10, 10).cumsum(axis=1), columns=[chr(65+i) for i in range(10)])
buffer = hv.streams.Buffer(data=initial_data, length=1000)

accumulated_data = initial_data.copy()

def create_stream_plot(data):
    return hv.NdOverlay(
        {i: hv.Curve(data, 'index', (chr(65+i), 'Amplitude')).opts(subcoordinate_y=True, tools=['hover']) for i in range(10)}, 'Channel'
    )
dmap_stream = hv.DynamicMap(create_stream_plot, streams=[buffer]).opts(width=800, axiswise=True, title='Streaming')

def create_inspection_plot(data):
    return hv.NdOverlay(
        {i: hv.Curve(data, 'index', (chr(65+i), 'Amplitude')).opts(subcoordinate_y=True, tools=['hover']) for i in range(10)}, 'Channel'
    ).opts(width=800, axiswise=True, shared_axes=False, title='Inspection')

inspection_plot = create_inspection_plot(initial_data)

# globals to control streaming state
streaming = False
paused = False
counter = 2

def stream_data():
    global streaming, counter, accumulated_data
    while streaming:
        if not paused:
            new_data = pd.DataFrame(np.random.randn(10, 10).cumsum(axis=1), columns=[chr(65+j) for j in range(10)], index=np.arange(counter*10, counter*10+10))
            new_data = new_data.reset_index()
            accumulated_data = pd.concat([accumulated_data, new_data])
            buffer.event(data=new_data)
            counter += 1
            dmap_stream.opts(xlim=(0, counter*10))
        time.sleep(0.07)


def start_stream(event):
    global streaming, paused, counter, accumulated_data
    if not streaming:
        buffer.clear()
        accumulated_data = pd.DataFrame(np.zeros((1, 10)), columns=[chr(65+i) for i in range(10)]).reset_index()
        buffer.event(data=accumulated_data)
        counter = 2  # Reset counter
        streaming = True
        paused = False
        thread = Thread(target=stream_data)
        thread.start()
    else:
        paused = False

def pause_stream(event):
    global paused
    paused = True

def stop_stream(event):
    global streaming, paused, accumulated_data
    streaming = False
    paused = False
    buffer.clear()  # Clear the buffer when stopping
    accumulated_data = pd.DataFrame(np.zeros((1, 10)), columns=[chr(65+i) for i in range(10)]).reset_index()
    buffer.event(data=accumulated_data)

def inspect_plot(event):
    def update_inspection_plot():
        global inspection_plot
        inspection_data = accumulated_data.copy()
        inspection_plot = create_inspection_plot(inspection_data)
        right_plot_panel.object = inspection_plot

    inspect_thread = Thread(target=update_inspection_plot)
    inspect_thread.start()

# Buttons!
start_button = pn.widgets.Button(name='Start Streaming', button_type='primary')
pause_button = pn.widgets.Button(name='Pause Streaming', button_type='warning')
stop_button = pn.widgets.Button(name='Stop Streaming', button_type='danger')
inspect_button = pn.widgets.Button(name='Inspect', button_type='success')

start_button.on_click(start_stream)
pause_button.on_click(pause_stream)
stop_button.on_click(stop_stream)
inspect_button.on_click(inspect_plot)

right_plot_panel = pn.pane.HoloViews(inspection_plot)
layout = pn.Column(
    pn.Row(start_button, pause_button, stop_button, inspect_button),
    pn.Row(dmap_stream, right_plot_panel)
)

layout.servable()
